#Reduced Dataset

## python imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt  # To visualize
import pandas as pd  # To read data
from sklearn import linear_model, metrics
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

##Mount Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Reading Data

In [3]:
data = pd.read_csv('/content/drive/MyDrive/AV_V/all_and_classification.csv')  # load data set


data = data.drop('run', axis=1)
data['forks'] = data['forks'].replace(['steady state'], 1)
data['forks'] = data['forks'].replace(['no steady state'], 0)
data = data.loc[data['forks'] == 1]
data = data.drop('forks', axis=1)
data = data.loc[data['steady_state_starts']<= 625]
data


,0,1,2,3,4,5,6,7,8,9,...,2992,2993,2994,2995,2996,2997,2998,2999,steady_state_starts,file_name
1,0.188219,0.075891,0.073269,0.073335,0.073138,0.074449,0.104530,0.079888,0.071762,0.072090,...,0.073138,0.073204,0.073859,0.073204,0.073466,0.073269,0.073400,0.073597,480,eclipse__eclipse-collections#org.eclipse.colle...
2,0.190317,0.075170,0.072417,0.072221,0.073073,0.073531,0.113967,0.096076,0.088605,0.087949,...,0.076218,0.074908,0.074908,0.074842,0.073925,0.075497,0.073925,0.074514,421,eclipse__eclipse-collections#org.eclipse.colle...
4,0.185860,0.074908,0.072155,0.072942,0.072810,0.074252,0.104530,0.095683,0.087753,0.085721,...,0.072745,0.076349,0.072745,0.072614,0.074056,0.072417,0.071959,0.073925,385,eclipse__eclipse-collections#org.eclipse.colle...
6,0.191103,0.077136,0.074187,0.073531,0.072614,0.074121,0.117703,0.094700,0.088146,0.085656,...,0.076808,0.076481,0.073794,0.075366,0.073662,0.074908,0.074187,0.073531,408,eclipse__eclipse-collections#org.eclipse.colle...
7,0.188219,0.074646,0.072876,0.073073,0.073925,0.074252,0.090472,0.094634,0.087359,0.086114,...,0.074187,0.074514,0.075104,0.075104,0.074514,0.075563,0.074711,0.074646,433,eclipse__eclipse-collections#org.eclipse.colle...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5848,0.000042,0.000024,0.000031,0.000031,0.000031,0.000031,0.000031,0.000031,0.000031,0.000031,...,0.000017,0.000017,0.000018,0.000018,0.000018,0.000018,0.000018,0.000018,53,apache__logging-log4j2#org.apache.logging.log4...
5849,0.000045,0.000023,0.000031,0.000030,0.000030,0.000030,0.000030,0.000030,0.000030,0.000030,...,0.000017,0.000017,0.000017,0.000018,0.000017,0.000017,0.000017,0.000017,321,apache__logging-log4j2#org.apache.logging.log4...
5851,0.001629,0.002516,0.002812,0.002784,0.002766,0.002760,0.003237,0.001095,0.001196,0.001301,...,0.001093,0.001142,0.001118,0.001779,0.002299,0.001127,0.001089,0.001095,395,yellowstonegames__SquidLib#squidpony.performan...
5855,0.001660,0.002597,0.002875,0.002770,0.002834,0.002764,0.002709,0.001095,0.001221,0.001445,...,0.001089,0.001090,0.001092,0.001093,0.001092,0.001106,0.001109,0.001106,190,yellowstonegames__SquidLib#squidpony.performan...


##Dividing data (train and test) based on file name 

In [4]:
data_file_name = data['file_name']

In [5]:
data_file_name_unique = data_file_name.drop_duplicates(keep="first")

In [6]:
from sklearn.model_selection import train_test_split
data_file_name_train, data_file_name_test = train_test_split(data_file_name_unique, test_size=0.3,
                                                    random_state=1)

In [7]:
train_data = pd.merge(data, data_file_name_train, left_on='file_name', right_on='file_name')

In [8]:
test_data = pd.merge(data, data_file_name_test, left_on='file_name', right_on='file_name')

In [9]:
train_data = train_data.drop('file_name', axis=1)
test_data = test_data.drop('file_name', axis=1)

In [10]:
X_train = train_data.drop('steady_state_starts', axis=1) 
y_train  = train_data['steady_state_starts']
X_test = test_data.drop('steady_state_starts', axis=1)
y_test = test_data['steady_state_starts']

### Standardizing the data

In [11]:
sc = StandardScaler()
sc.fit(X_train.T)
X_train = pd.DataFrame(sc.transform(X_train.T))
X_train = X_train.T

In [12]:
sc = StandardScaler()
sc.fit(X_test.T)
X_test = pd.DataFrame(sc.transform(X_test.T))
X_test = X_test.T

##Finding best hyperparameter values

In [13]:
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten

In [14]:
model = Sequential()
model.add(Conv1D(32, 2, activation="softplus", input_shape=(3000, 1)))
model.add(Flatten())
model.add(Dense(64, activation="softplus"))
model.add(Dense(1))
model.compile(loss="mse", optimizer="adam")
model.summary()

xtrain = X_train.values.reshape(X_train.shape[0], X_train.shape[1], 1)
xtest = X_test.values.reshape(X_test.shape[0], X_test.shape[1], 1)

model.fit(xtrain, y_train, batch_size=12,epochs=20, verbose=0)
y_pred_cnn = model.predict(xtest)

cnn_mae = mean_absolute_error(y_test, y_pred_cnn)
cnn_mse = mean_squared_error(y_test, y_pred_cnn)
cnn_rmse = mean_squared_error(y_test, y_pred_cnn, squared=False)
cnn_r2 = r2_score(y_test, y_pred_cnn)

print("MAE:", cnn_mae)
print("MSE:", cnn_mse)
print("RMSE:", cnn_rmse)
print("R2 Score:", cnn_r2)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 2999, 32)          96        
                                                                 
 flatten (Flatten)           (None, 95968)             0         
                                                                 
 dense (Dense)               (None, 64)                6142016   
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 6,142,177
Trainable params: 6,142,177
Non-trainable params: 0
_________________________________________________________________
39/39 [==============================] - 2s 55ms/step
MAE: 99.1069825924036
MSE: 31269.36674727732
RMSE: 176.8314642456973
R2 Score: -0.415184223798315


In [15]:
model = Sequential()
model.add(Conv1D(32, 2, activation="softsign", input_shape=(3000, 1)))
model.add(Flatten())
model.add(Dense(64, activation="softsign"))
model.add(Dense(1))
model.compile(loss="mse", optimizer="adam")
model.summary()

xtrain = X_train.values.reshape(X_train.shape[0], X_train.shape[1], 1)
xtest = X_test.values.reshape(X_test.shape[0], X_test.shape[1], 1)

model.fit(xtrain, y_train, batch_size=12,epochs=20, verbose=0)
y_pred_cnn = model.predict(xtest)

cnn_mae = mean_absolute_error(y_test, y_pred_cnn)
cnn_mse = mean_squared_error(y_test, y_pred_cnn)
cnn_rmse = mean_squared_error(y_test, y_pred_cnn, squared=False)
cnn_r2 = r2_score(y_test, y_pred_cnn)

print("MAE:", cnn_mae)
print("MSE:", cnn_mse)
print("RMSE:", cnn_rmse)
print("R2 Score:", cnn_r2)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_1 (Conv1D)           (None, 2999, 32)          96        
                                                                 
 flatten_1 (Flatten)         (None, 95968)             0         
                                                                 
 dense_2 (Dense)             (None, 64)                6142016   
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 6,142,177
Trainable params: 6,142,177
Non-trainable params: 0
_________________________________________________________________
39/39 [==============================] - 1s 25ms/step
MAE: 115.17071823438519
MSE: 22135.44186120149
RMSE: 148.77984359852476
R2 Score: -0.0018024465271508916


In [16]:
model = Sequential()
model.add(Conv1D(32, 2, activation="tanh", input_shape=(3000, 1)))
model.add(Flatten())
model.add(Dense(64, activation="tanh"))
model.add(Dense(1))
model.compile(loss="mse", optimizer="adam")
model.summary()

xtrain = X_train.values.reshape(X_train.shape[0], X_train.shape[1], 1)
xtest = X_test.values.reshape(X_test.shape[0], X_test.shape[1], 1)

model.fit(xtrain, y_train, batch_size=12,epochs=20, verbose=0)
y_pred_cnn = model.predict(xtest)

cnn_mae = mean_absolute_error(y_test, y_pred_cnn)
cnn_mse = mean_squared_error(y_test, y_pred_cnn)
cnn_rmse = mean_squared_error(y_test, y_pred_cnn, squared=False)
cnn_r2 = r2_score(y_test, y_pred_cnn)

print("MAE:", cnn_mae)
print("MSE:", cnn_mse)
print("RMSE:", cnn_rmse)
print("R2 Score:", cnn_r2)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_2 (Conv1D)           (None, 2999, 32)          96        
                                                                 
 flatten_2 (Flatten)         (None, 95968)             0         
                                                                 
 dense_4 (Dense)             (None, 64)                6142016   
                                                                 
 dense_5 (Dense)             (None, 1)                 65        
                                                                 
Total params: 6,142,177
Trainable params: 6,142,177
Non-trainable params: 0
_________________________________________________________________
39/39 [==============================] - 1s 28ms/step
MAE: 114.87156957684512
MSE: 22147.295517324248
RMSE: 148.8196744967689
R2 Score: -0.002338917485288139


In [17]:

model = Sequential()
model.add(Conv1D(32, 2, activation="sigmoid", input_shape=(3000, 1)))
model.add(Flatten())
model.add(Dense(64, activation="sigmoid"))
model.add(Dense(1))
model.compile(loss="mse", optimizer="adam")
model.summary()

xtrain = X_train.values.reshape(X_train.shape[0], X_train.shape[1], 1)
xtest = X_test.values.reshape(X_test.shape[0], X_test.shape[1], 1)

model.fit(xtrain, y_train, batch_size=12,epochs=20, verbose=0)
y_pred_cnn = model.predict(xtest)

cnn_mae = mean_absolute_error(y_test, y_pred_cnn)
cnn_mse = mean_squared_error(y_test, y_pred_cnn)
cnn_rmse = mean_squared_error(y_test, y_pred_cnn, squared=False)
cnn_r2 = r2_score(y_test, y_pred_cnn)

print("MAE:", cnn_mae)
print("MSE:", cnn_mse)
print("RMSE:", cnn_rmse)
print("R2 Score:", cnn_r2)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_3 (Conv1D)           (None, 2999, 32)          96        
                                                                 
 flatten_3 (Flatten)         (None, 95968)             0         
                                                                 
 dense_6 (Dense)             (None, 64)                6142016   
                                                                 
 dense_7 (Dense)             (None, 1)                 65        
                                                                 
Total params: 6,142,177
Trainable params: 6,142,177
Non-trainable params: 0
_________________________________________________________________
39/39 [==============================] - 1s 30ms/step
MAE: 113.41185317184913
MSE: 22225.698271418143
RMSE: 149.08285706753188
R2 Score: -0.005887257353913045


In [18]:
model = Sequential()
model.add(Conv1D(32, 2, activation="hard_sigmoid", input_shape=(3000, 1)))
model.add(Flatten())
model.add(Dense(64, activation="hard_sigmoid"))
model.add(Dense(1))
model.compile(loss="mse", optimizer="adam")
model.summary()

xtrain = X_train.values.reshape(X_train.shape[0], X_train.shape[1], 1)
xtest = X_test.values.reshape(X_test.shape[0], X_test.shape[1], 1)

model.fit(xtrain, y_train, batch_size=12,epochs=20, verbose=0)
y_pred_cnn = model.predict(xtest)

cnn_mae = mean_absolute_error(y_test, y_pred_cnn)
cnn_mse = mean_squared_error(y_test, y_pred_cnn)
cnn_rmse = mean_squared_error(y_test, y_pred_cnn, squared=False)
cnn_r2 = r2_score(y_test, y_pred_cnn)

print("MAE:", cnn_mae)
print("MSE:", cnn_mse)
print("RMSE:", cnn_rmse)
print("R2 Score:", cnn_r2)

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_4 (Conv1D)           (None, 2999, 32)          96        
                                                                 
 flatten_4 (Flatten)         (None, 95968)             0         
                                                                 
 dense_8 (Dense)             (None, 64)                6142016   
                                                                 
 dense_9 (Dense)             (None, 1)                 65        
                                                                 
Total params: 6,142,177
Trainable params: 6,142,177
Non-trainable params: 0
_________________________________________________________________
39/39 [==============================] - 1s 29ms/step
MAE: 111.84201608232281
MSE: 22352.628975255426
RMSE: 149.50795622727048
R2 Score: -0.011631867759295922


#Full Dataset

## Reading Data

In [19]:
data = pd.read_csv('/content/drive/MyDrive/AV_V/all_and_classification.csv')  # load data set


data = data.drop('run', axis=1)
data['forks'] = data['forks'].replace(['steady state'], 1)
data['forks'] = data['forks'].replace(['no steady state'], 0)
data = data.loc[data['forks'] == 1]
data = data.drop('forks', axis=1)


##Dividing data (train and test) based on file name 

In [20]:
data_file_name = data['file_name']


In [21]:
data_file_name_unique = data_file_name.drop_duplicates(keep="first")

In [22]:
from sklearn.model_selection import train_test_split
data_file_name_train, data_file_name_test = train_test_split(data_file_name_unique, test_size=0.3,
                                                    random_state=1)

### Standardizing the data

In [23]:
data_without_file_name_and_y = data.drop('file_name', axis=1)
data_without_file_name_and_y = data_without_file_name_and_y.drop('steady_state_starts', axis=1)
sc = StandardScaler()
sc.fit(data_without_file_name_and_y.T)
data_std = pd.DataFrame(sc.transform(data_without_file_name_and_y.T))
data_std = data_std.T

In [24]:
data_std = data_std.merge(data_file_name, left_index=True, right_index=True)
data_std = data_std.merge(data['steady_state_starts'], left_index=True, right_index=True)

In [25]:
train_data = pd.merge(data_std, data_file_name_train, left_on='file_name', right_on='file_name')

In [26]:
test_data = pd.merge(data_std, data_file_name_test, left_on='file_name', right_on='file_name')

In [27]:
train_data = train_data.drop('file_name', axis=1)
test_data = test_data.drop('file_name', axis=1)

In [28]:
X_train = train_data.drop('steady_state_starts', axis=1) 
y_train  = train_data['steady_state_starts']
X_test = test_data.drop('steady_state_starts', axis=1)
y_test = test_data['steady_state_starts']

##Finding best hyperparameter values

In [29]:
model = Sequential()
model.add(Conv1D(32, 2, activation="softplus", input_shape=(3000, 1)))
model.add(Flatten())
model.add(Dense(64, activation="softplus"))
model.add(Dense(1))
model.compile(loss="mse", optimizer="adam")
model.summary()

xtrain = X_train.values.reshape(X_train.shape[0], X_train.shape[1], 1)
xtest = X_test.values.reshape(X_test.shape[0], X_test.shape[1], 1)

model.fit(xtrain, y_train, batch_size=12,epochs=20, verbose=0)
y_pred_cnn = model.predict(xtest)

cnn_mae = mean_absolute_error(y_test, y_pred_cnn)
cnn_mse = mean_squared_error(y_test, y_pred_cnn)
cnn_rmse = mean_squared_error(y_test, y_pred_cnn, squared=False)
cnn_r2 = r2_score(y_test, y_pred_cnn)

print("MAE:", cnn_mae)
print("MSE:", cnn_mse)
print("RMSE:", cnn_rmse)
print("R2 Score:", cnn_r2)

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_5 (Conv1D)           (None, 2999, 32)          96        
                                                                 
 flatten_5 (Flatten)         (None, 95968)             0         
                                                                 
 dense_10 (Dense)            (None, 64)                6142016   
                                                                 
 dense_11 (Dense)            (None, 1)                 65        
                                                                 
Total params: 6,142,177
Trainable params: 6,142,177
Non-trainable params: 0
_________________________________________________________________
42/42 [==============================] - 4s 95ms/step
MAE: 499.07875080201245
MSE: 591829.9030836078
RMSE: 769.3048180556311
R2 Score: -0.3228267379487324


In [30]:
model = Sequential()
model.add(Conv1D(32, 2, activation="softsign", input_shape=(3000, 1)))
model.add(Flatten())
model.add(Dense(64, activation="softsign"))
model.add(Dense(1))
model.compile(loss="mse", optimizer="adam")
model.summary()

xtrain = X_train.values.reshape(X_train.shape[0], X_train.shape[1], 1)
xtest = X_test.values.reshape(X_test.shape[0], X_test.shape[1], 1)

model.fit(xtrain, y_train, batch_size=12,epochs=20, verbose=0)
y_pred_cnn = model.predict(xtest)

cnn_mae = mean_absolute_error(y_test, y_pred_cnn)
cnn_mse = mean_squared_error(y_test, y_pred_cnn)
cnn_rmse = mean_squared_error(y_test, y_pred_cnn, squared=False)
cnn_r2 = r2_score(y_test, y_pred_cnn)

print("MAE:", cnn_mae)
print("MSE:", cnn_mse)
print("RMSE:", cnn_rmse)
print("R2 Score:", cnn_r2)

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_6 (Conv1D)           (None, 2999, 32)          96        
                                                                 
 flatten_6 (Flatten)         (None, 95968)             0         
                                                                 
 dense_12 (Dense)            (None, 64)                6142016   
                                                                 
 dense_13 (Dense)            (None, 1)                 65        
                                                                 
Total params: 6,142,177
Trainable params: 6,142,177
Non-trainable params: 0
_________________________________________________________________
42/42 [==============================] - 1s 25ms/step
MAE: 425.3738277111253
MSE: 463035.6981069743
RMSE: 680.4672645373723
R2 Score: -0.034952777629644105


In [31]:
model = Sequential()
model.add(Conv1D(32, 2, activation="tanh", input_shape=(3000, 1)))
model.add(Flatten())
model.add(Dense(64, activation="tanh"))
model.add(Dense(1))
model.compile(loss="mse", optimizer="adam")
model.summary()

xtrain = X_train.values.reshape(X_train.shape[0], X_train.shape[1], 1)
xtest = X_test.values.reshape(X_test.shape[0], X_test.shape[1], 1)

model.fit(xtrain, y_train, batch_size=12,epochs=20, verbose=0)
y_pred_cnn = model.predict(xtest)

cnn_mae = mean_absolute_error(y_test, y_pred_cnn)
cnn_mse = mean_squared_error(y_test, y_pred_cnn)
cnn_rmse = mean_squared_error(y_test, y_pred_cnn, squared=False)
cnn_r2 = r2_score(y_test, y_pred_cnn)

print("MAE:", cnn_mae)
print("MSE:", cnn_mse)
print("RMSE:", cnn_rmse)
print("R2 Score:", cnn_r2)

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_7 (Conv1D)           (None, 2999, 32)          96        
                                                                 
 flatten_7 (Flatten)         (None, 95968)             0         
                                                                 
 dense_14 (Dense)            (None, 64)                6142016   
                                                                 
 dense_15 (Dense)            (None, 1)                 65        
                                                                 
Total params: 6,142,177
Trainable params: 6,142,177
Non-trainable params: 0
_________________________________________________________________
42/42 [==============================] - 1s 28ms/step
MAE: 403.45078346946025
MSE: 482949.7457810946
RMSE: 694.9458581652923
R2 Score: -0.07946359836860672


In [32]:

model = Sequential()
model.add(Conv1D(32, 2, activation="sigmoid", input_shape=(3000, 1)))
model.add(Flatten())
model.add(Dense(64, activation="sigmoid"))
model.add(Dense(1))
model.compile(loss="mse", optimizer="adam")
model.summary()

xtrain = X_train.values.reshape(X_train.shape[0], X_train.shape[1], 1)
xtest = X_test.values.reshape(X_test.shape[0], X_test.shape[1], 1)

model.fit(xtrain, y_train, batch_size=12,epochs=20, verbose=0)
y_pred_cnn = model.predict(xtest)

cnn_mae = mean_absolute_error(y_test, y_pred_cnn)
cnn_mse = mean_squared_error(y_test, y_pred_cnn)
cnn_rmse = mean_squared_error(y_test, y_pred_cnn, squared=False)
cnn_r2 = r2_score(y_test, y_pred_cnn)

print("MAE:", cnn_mae)
print("MSE:", cnn_mse)
print("RMSE:", cnn_rmse)
print("R2 Score:", cnn_r2)

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_8 (Conv1D)           (None, 2999, 32)          96        
                                                                 
 flatten_8 (Flatten)         (None, 95968)             0         
                                                                 
 dense_16 (Dense)            (None, 64)                6142016   
                                                                 
 dense_17 (Dense)            (None, 1)                 65        
                                                                 
Total params: 6,142,177
Trainable params: 6,142,177
Non-trainable params: 0
_________________________________________________________________
42/42 [==============================] - 1s 31ms/step
MAE: 395.3981762586159
MSE: 495476.2109796101
RMSE: 703.9007110236572
R2 Score: -0.10746208747881258


In [33]:
model = Sequential()
model.add(Conv1D(32, 2, activation="hard_sigmoid", input_shape=(3000, 1)))
model.add(Flatten())
model.add(Dense(64, activation="hard_sigmoid"))
model.add(Dense(1))
model.compile(loss="mse", optimizer="adam")
model.summary()

xtrain = X_train.values.reshape(X_train.shape[0], X_train.shape[1], 1)
xtest = X_test.values.reshape(X_test.shape[0], X_test.shape[1], 1)

model.fit(xtrain, y_train, batch_size=12,epochs=20, verbose=0)
y_pred_cnn = model.predict(xtest)

cnn_mae = mean_absolute_error(y_test, y_pred_cnn)
cnn_mse = mean_squared_error(y_test, y_pred_cnn)
cnn_rmse = mean_squared_error(y_test, y_pred_cnn, squared=False)
cnn_r2 = r2_score(y_test, y_pred_cnn)

print("MAE:", cnn_mae)
print("MSE:", cnn_mse)
print("RMSE:", cnn_rmse)
print("R2 Score:", cnn_r2)

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_9 (Conv1D)           (None, 2999, 32)          96        
                                                                 
 flatten_9 (Flatten)         (None, 95968)             0         
                                                                 
 dense_18 (Dense)            (None, 64)                6142016   
                                                                 
 dense_19 (Dense)            (None, 1)                 65        
                                                                 
Total params: 6,142,177
Trainable params: 6,142,177
Non-trainable params: 0
_________________________________________________________________
42/42 [==============================] - 1s 31ms/step
MAE: 392.16443034824425
MSE: 502728.02881902823
RMSE: 709.0331648230767
R2 Score: -0.12367096520995458
